In [5]:
'''
Read NWM output and USGS data, process monthly and daily volume
time series, store for subsequent evaluation and plotting

Note: This script assumes csv files have been generated that contain
NWM AnA hourly channel-rt output times series and USGS (native time step)
per specified location. Code would require altering to read time series
from alternative source or format.
'''
import pandas as pd
import numpy as np
import os

### base working directory
base_dir = 'C:\\repos\\git\\nwm_eval_projects\\annual_runoff'

### define list of sites to include in analysis by USGS ID
site_list_fname = 'Volumetric_Eval_Sites_3.csv'
site_id_header = 'USGS_ID'
site_num_header = 'Site_Num' # numbered roughly from East to West

### NWM directory (location of hourly AnA time series csv files)
nwm_dir = 'ts-data\\nwm_ana'
nwm_nheadlines = 19

### NWM gap summary and aggregated time series output files
nwm_gapsum_fname = "NWM_gaps.csv"
nwm_gaplist_fname = "NWM_gaps_list.csv"
nwm_accum19_fname = "NWM_DailyAccum_WY19.csv"
nwm_accum20_fname = "NWM_DailyAccum_WY20.csv"
nwm_monthly_fname = "NWM_MonthlyVolume_WYs19-20.csv"

### USGS data directory (raw USGS data csv files)
usgs_dir = 'ts-data\\usgs'
usgs_nheadlines = 21

### USGS gap summary and aggregated time series output files
usgs_gapsum_fname = "USGS_gaps.csv"
usgs_gaplist_fname = "USGS_gaps_list.csv"
usgs_accum19_fname = "USGS_DailyAccum_WY19.csv"
usgs_accum20_fname = "USGS_DailyAccum_WY20.csv"
usgs_monthly_fname = "USGS_MonthlyVolume_WYs19-20.csv"

In [6]:
### read analysis site information
### sites selected by external review, stored in a csv table

read_path = os.path.join(base_dir, site_list_fname)
sites_df = pd.read_csv(read_path)
site_ids =sites_df[site_id_header].astype("string")
for i in range(len(site_ids)):
    site_ids[i] = site_ids[i].zfill(8) #pad string with zeros to 8 digits
sites_df[site_id_header] = site_ids
sites_df = sites_df.set_index('Site_Num')

In [7]:
def read_csv_to_df_dict(read_path, nheadlines, sitelist, q_unit):
    '''
    read all output/data time series in csv files in read_path 
    store in dictionary of dataframes if location is in sitelist 
    return a dictionary containing all data to be processed and plotted
    '''
    tsdict = {}
    q_unit = 'q_' + q_unit
    for fname in os.listdir(read_path):
        if fname.lower().endswith('.csv'):
            # get the location from the filename
            extract_start = fname.find("_")+1
            location = fname[extract_start:fname.find("_", extract_start+1)]
            if not location.isdigit():
                continue
            if not location in sitelist.values:
                continue
            fpath = os.path.join(read_path, fname)
            df = pd.read_csv(fpath, names=['datetime', q_unit], index_col = 'datetime', skiprows=nheadlines, parse_dates=True)
            tsdict[location] = df
    return tsdict

In [8]:
def process_data(sites_df, q_unit, tsdict, write_path, filenames):
    '''
    loop through all sites 
    check for missing data (>60 min gap), write to file max gap size per location 
    calculate average daily flow, fill NAN by interpolation, calculate cumulative daily volume by year
    '''
    # itialize gaplist dataframe
    gapstats = sites_df.iloc[:,:1].copy()
    gapstats['n_gaps'] = 0
    gapstats['tot_days'] = 0
    gapstats['max_gap_days'] = 0
    gapstats = gapstats.set_index('USGS_ID')
    site_ids = sites_df['USGS_ID']

    if q_unit == 'cfs':
        q_to_AF = 43560.000443512
    else:
        q_to_AF = 1233.48185532
    
    q_unit = 'q_' + q_unit
    
    for location, ts in tsdict.items():

        # remove rows with missing data (-999999 or other value < 0)
        ts = ts.loc[ts[q_unit] >= 0]
        
        # calculate deltas between each time step
        diff = np.diff(ts.index) 
        secs = diff / np.timedelta64(1, 's')
        minutes = secs / 60
        hrs = secs / 3600
        days = hrs / 24
        ts_with_timedelta = ts[1:].copy() # copy original dataframe beginning at 2nd index as deltas have one less entry
        ts_with_timedelta['gap_minutes'] = minutes
        ts_with_timedelta['gap_hours'] = hrs
        ts_with_timedelta['gap_days'] = days

        # find dates with data gaps (delta > 60 min)
        dates_w_gaps = ts_with_timedelta[ts_with_timedelta['gap_minutes'].gt(60)].index
        gaplist = ts_with_timedelta.loc[dates_w_gaps].copy()
        gaplist.insert(0,'prior_q',gaplist[q_unit]) 
        gaplist.insert(0,'ID', location)

        # store max gap size per location in gapstats 
        gapstats.loc[location, :]  = [len(dates_w_gaps), gaplist['gap_days'].sum(axis = 0), gaplist['gap_days'].max(axis = 0)]
        # list each gap per location, with surrounding values and gapsize in ts_w_gaps 
        for i in range(len(dates_w_gaps)):
            ind = np.where(ts_with_timedelta.index==dates_w_gaps[i])
            to_col = gaplist.columns.get_loc('prior_q')
            from_col = ts_with_timedelta.columns.get_loc(q_unit) 
            gaplist.iat[i,to_col] = ts_with_timedelta.iat[int(ind[0]) - 1, from_col]

        # calculate average daily flow, fill NAN by interpolation
        # calculate monthly volume and cumulative daily volume time series
        ts_daily_mean = ts_with_timedelta[q_unit].resample('D').mean()  # get mean daily flow
        ts_daily_df = ts_daily_mean.to_frame() # write to dataframe
        ts_daily_df['interp'] = ts_daily_mean.interpolate() # fill NAN values by interpolation
        ts_daily_df['vol_AF'] = ts_daily_df['interp'] * 3600 * 24 / q_to_AF # add column of daily volume in AF  
        ts_daily_df['cumvol_AF'] = ts_daily_df['vol_AF'].cumsum() # add column of cumulative sum of daily volume
        ts_monthly = ts_daily_df['vol_AF'].resample('M').sum() # get total monthly volumes

        wy19 = ts_daily_df.loc[:'2019-09-30'].copy()
        wy20 = ts_daily_df.loc['2019-10-01':].copy()
        wy20['cumvol_AF'] = wy20['vol_AF'].cumsum()

        # if first site, initialize dataframes of processed results
        # otherwise append subsequent sites's results as new columns
        
        if location == site_ids.iloc[0]:
            gaplist_allsites = gaplist.copy()
            wy19_dailyaccum_allsites = pd.DataFrame(wy19['cumvol_AF'].values, columns = [location], index = wy19.index)
            wy20_dailyaccum_allsites = pd.DataFrame(wy20['cumvol_AF'].values, columns = [location], index = wy20.index)
            monthly_allsites = pd.DataFrame(ts_monthly.values, columns = [location], index = ts_monthly.index)
            monthly_allsites.index = monthly_allsites.index.strftime("%b %Y")
        else:
            gaplist_allsites = gaplist_allsites.append(gaplist)    
            wy19_dailyaccum_allsites[location] = wy19['cumvol_AF'].values
            wy20_dailyaccum_allsites[location] = wy20['cumvol_AF'].values
            monthly_allsites[location] = ts_monthly.values

    gaplist_allsites.to_csv(os.path.join(write_path, filenames[0]))
    gapstats.to_csv(os.path.join(write_path, filenames[1]))

    wy19_dailyaccum_allsites.to_csv(os.path.join(write_path, filenames[2]))
    wy20_dailyaccum_allsites.to_csv(os.path.join(write_path, filenames[3]))
    monthly_allsites.to_csv(os.path.join(write_path, filenames[4]))  
            

In [9]:
### Read NWM
nwm_path = os.path.join(base_dir, nwm_dir)
nwm_tsdict = read_csv_to_df_dict(nwm_path, nwm_nheadlines, site_ids, 'cms')

## Process NWM
nwm_filenames = [nwm_gaplist_fname, nwm_gapsum_fname, nwm_accum19_fname, nwm_accum20_fname, nwm_monthly_fname]
process_data(sites_df, 'cms', nwm_tsdict, nwm_path, nwm_filenames)

### Read USGS
usgs_path = os.path.join(base_dir, usgs_dir)
usgs_tsdict = read_csv_to_df_dict(usgs_path, usgs_nheadlines, site_ids, 'cfs')

## Process USGS
usgs_filenames = [usgs_gaplist_fname, usgs_gapsum_fname, usgs_accum19_fname, usgs_accum20_fname, usgs_monthly_fname]
process_data(sites_df, 'cfs', usgs_tsdict, usgs_path, usgs_filenames)